In [1]:
import re
import subprocess

import dxdata
import dxpy
import pandas as pd
import pyspark

from pyspark.sql import functions as F
from pyspark.conf import SparkConf
from pyspark.sql.types import StringType

from pathlib import Path

Path("../tmp").resolve().mkdir(parents=True, exist_ok=True)

trait = "metabolic"

In [2]:
conf = SparkConf()
conf.set("autoBroadcastJoinThreshold", -1)

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

dispensed_database_name = dxpy.find_one_data_object(
    classname="database", name="app*", folder="/", name_mode="glob", describe=True
)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)["id"]

spark.sql("USE " + dispensed_database_name)

dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]


def get_columns_to_keep(df, threshold=200) -> list:
    """
    This function drops all columns which contain more null values than threshold
    :param df: A PySpark DataFrame
    """
    null_counts = (
        df.select([F.count(F.when(~F.col(c).isNull(), c)).alias(c) for c in df.columns])
        .collect()[0]
        .asDict()
    )
    to_keep = [k for k, v in null_counts.items() if v > threshold]
    # df = df.select(*to_keep)

    return to_keep

In [3]:
def new_names(s: str) -> str:
    """
    Fixes a column header for PHESANT use
    """
    s = s.replace("p", "x").replace("i", "")

    if bool(re.search("_\d$", s)):
        s += "_0"
    else:
        s += "_0_0"
    return s

In [12]:
age_sex = "|".join(["31", "21022"])
age_sex_fields = list(
    participant.find_fields(lambda f: bool(re.match(f"^p({age_sex})$", f.name)))
)

In [34]:
metabolic = "|".join(
    [
        "21002",
        "50",
        "48",
        "21001",
        "23099",
        "23127",
        "102",
        "4080",
        "4079",
    ]
)

metabolic_fields = list(
    participant.find_fields(lambda f: bool(re.match(f"^p({metabolic})\D", f.name)))
)

field_names = (
    ["eid"]
    + [f.name for f in age_sex_fields]
    + [f.name for f in metabolic_fields]
)
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

[dxdata.dataset.dataset] WARNING: Dataframe schema name(s): ['participant_0001$eid', 'participant_0001$p31', 'participant_0042$p21022', 'participant_0001$p48_i0', 'participant_0001$p48_i1', 'participant_0001$p48_i2', 'participant_0001$p48_i3', 'participant_0001$p50_i0', 'participant_0001$p50_i1', 'participant_0001$p50_i2', 'participant_0001$p50_i3', 'participant_0002$p102_i0_a0', 'participant_0002$p102_i0_a1', 'participant_0002$p102_i1_a0', 'participant_0002$p102_i1_a1', 'participant_0002$p102_i2_a0', 'participant_0002$p102_i2_a1', 'participant_0002$p102_i3_a0', 'participant_0002$p102_i3_a1', 'participant_0008$p4079_i0_a0', 'participant_0008$p4079_i0_a1', 'participant_0008$p4079_i1_a0', 'participant_0008$p4079_i1_a1', 'participant_0008$p4079_i2_a0', 'participant_0008$p4079_i2_a1', 'participant_0008$p4079_i3_a0', 'participant_0008$p4079_i3_a1', 'participant_0008$p4080_i0_a0', 'participant_0008$p4080_i0_a1', 'participant_0008$p4080_i1_a0', 'participant_0008$p4080_i1_a1', 'participant_000

In [35]:
to_drop = [x for x in df.columns if "a1" in x]
df = df.drop(*to_drop)
colnames = [re.sub("_a\d", "", x) for x in df.columns]
colnames = ['xeid'] + [new_names(s) for s in colnames[1:]]
print(colnames[:10])

['xeid', 'x31_0_0', 'x21022_0_0', 'x48_0_0', 'x48_1_0', 'x48_2_0', 'x48_3_0', 'x50_0_0', 'x50_1_0', 'x50_2_0']


In [39]:
df = df.toDF(*colnames)

In [43]:
df.write.csv("/tmp/phenos.tsv", sep="\t", header=True, emptyValue='NA')

In [44]:
subprocess.run(
    ["hadoop", "fs", "-rm", "/tmp/phenos.tsv/_SUCCESS"], check=True, shell=False
)
subprocess.run(
    ["hadoop", "fs", "-get", "/tmp/phenos.tsv", "../tmp/phenos.tsv"],
    check=True,
    shell=False,
)

CompletedProcess(args=['hadoop', 'fs', '-get', '/tmp/phenos.tsv', '../tmp/phenos.tsv'], returncode=0)

In [45]:
!sed -e '3,${/^xeid/d' -e '}' ../tmp/phenos.tsv/part* > ../tmp/metabolic.QT.raw.tsv

In [46]:
# Upload to project

subprocess.run(
    ["dx", "upload", "../tmp/metabolic.QT.raw.tsv", "--path", "Data/phenotypes/"],
    check=True,
    shell=False,
)

CompletedProcess(args=['dx', 'upload', '../tmp/metabolic.QT.raw.tsv', '--path', 'Data/phenotypes/'], returncode=0)